In [73]:
import pandas as pd
import os

unformated_animal_data = "/home/spsalmon/better_paper_code/mortality_data/animals_and_plants/41586_2014_BFnature12789_MOESM42_ESM.xls"
output_dir = "/home/spsalmon/better_paper_code/mortality_data/animals_and_plants/data"
os.makedirs(output_dir, exist_ok=True)

print(os.path.dirname(output_dir))

# Use pandas to read the Excel file
xls = pd.ExcelFile(unformated_animal_data)

# Get the list of sheet names in the Excel file
sheet_names = xls.sheet_names

def process_sheet(xls, sheet_name):
    df = pd.read_excel(xls, sheet_name=sheet_name)
    
    # Extract information from the first line
    if len(df.columns) ==6:
        subphylum = df.columns[3]
        time_unit = df.columns[-1]
    elif len(df.columns) == 9:
        subphylum = df.columns[3]
        time_unit = df.columns[5]
    else:
        return "Not actual animal data"
    
    common_name = df.columns[0]
    latin_name = df.columns[1]

    save_name = f'{common_name} ({latin_name}).csv'

    # Replace the first line of the excel file with the actual column names
    new_columns = df.iloc[0].to_list()
    df.columns = new_columns
    df = df.drop(0)


    if "Nx" in df.columns:
        for i in range(1,len(df['Nx'])):
            if df['Nx'].iloc[i] > df['Nx'].iloc[i-1]:
                return "Excluded because Nx is increasing"
        df['lx'] = df['Nx']/df['Nx'].iloc[0]

    elif "lx" in df.columns:
        for i in range(1,len(df['lx'])):
            if df['lx'].iloc[i] > df['lx'].iloc[i-1]:
                return "Excluded because lx is increasing"
            if df['lx'].iloc[0] < 1:
                return "Excluded because lx[0] < 1"
        df['lx'] = df['lx']/df['lx'].iloc[0]
    
    elif "Lx" in df.columns:
        for i in range(1,len(df['Lx'])):
            if df['Lx'].iloc[i] > df['Lx'].iloc[i-1]:
                return "Excluded because Lx is increasing"
        if df['Lx'].iloc[0] < 1:
                return "Excluded because Lx[0] < 1"
        df['lx'] = df['Lx']/df['Lx'].iloc[0]
    
    if "lx" not in df.columns:
        return "Excluded because no way of calculating lx (can't cut data when only 5% of population is left)"

    df = df.loc[df['lx'] >= 0.05]

    if 'qx' not in df.columns:
        df['px'] = df['lx'].shift(-1)/df['lx']
        df['qx'] = 1 - df['px']

    zero_qx = (df['qx'] == 0).sum()
    ratio_zero_qx = zero_qx/len(df['qx'])
    if ratio_zero_qx > 0.2:
        return "Excluded because the ratio of zero qx time points is > 0.2"
    
    df = df[['x', 'qx']]
    df['qx'] = df['qx'].replace(0, 1e-5)

    if len(df['x']) <= 3:
        return "Excluded because less than 3 time points"
    
    df.rename(columns={'x':'Age'}, inplace=True)
    df.dropna(inplace=True)
    df['time_unit'] = time_unit
    df['subphylum'] = subphylum
    df.to_csv(os.path.join(output_dir, f'{save_name}'), index=False)
    return None
    
# Iterate through each sheet and perform desired operations
open(os.path.join(os.path.dirname(output_dir), 'excluded_data.txt'), 'w').close()
for sheet_name in sheet_names[1:]:
    p = process_sheet(xls, sheet_name)
    if p is not None and p != "Not actual animal data":
        with open(os.path.join(os.path.dirname(output_dir), 'excluded_data.txt'), 'a') as f:
            f.write(f'{sheet_name} : {p}\n')

    

/home/spsalmon/better_paper_code/mortality_data/animals_and_plants
